In [1]:
import pandas as pd
import numpy as np
import os

# Importing data

In [2]:
#read a txt file and convert it to a dataframe
def read_txt(path):
    df = pd.read_csv(path, sep='\t', header=None)
    return df

In [3]:
# import the data
plots = pd.read_csv('MovieSummaries/plot_summaries.txt',header=None, sep="\t")
movies = pd.read_csv('MovieSummaries/movies_metadata.tsv',header=None, sep="\t")
characters = pd.read_csv('MovieSummaries/characters_metadata.tsv',header=None, sep="\t")
names = pd.read_csv('MovieSummaries/names_clusters.txt',header=None, sep="\t")
tvtropes = pd.read_csv('MovieSummaries/tvtropes_clusters.txt',header=None, sep="\t")

In [4]:
plots.columns = ['WikiID', 'Plot']

`movies` data

1. Wikipedia movie ID
2. Freebase movie ID
3. Movie name
4. Movie release date
5. Movie box office revenue
6. Movie runtime
7. Movie languages (Freebase ID:name tuples)
8. Movie countries (Freebase ID:name tuples)
9. Movie genres (Freebase ID:name tuples)

In [5]:
movies.columns = ['WikiID', 'FreebaseID', 'Name', 'ReleaseDate', 'Revenue', 'Runtime', 'Languages', 'Countries', 'Genres']

In [6]:
# extracting the years from the release date feature
movie_with_date = movies[-movies["ReleaseDate"].isna()].copy(deep=True)
dates = movie_with_date["ReleaseDate"]
date_years = dates.astype(str).str.extract(r'(\d{4})')

`characters` data:
1. Wikipedia movie ID
2. Freebase movie ID
3. Movie release date
4. Character name
5. Actor date of birth
6. Actor gender
7. Actor height (in meters)
8. Actor ethnicity (Freebase ID)
9. Actor name
10. Actor age at movie release
11. Freebase character/actor map ID
12. Freebase character ID
13. Freebase actor ID

In [7]:
characters.columns = ['WikiID', 'FreebaseID', 'ReleaseDate', 'CharacterName', 'DateOfBirth', 'Gender', 'Height', 'Ethnicity', 'Name', 'AgeAtRealease', 'FreebaseCharacterActorMapID', 'FreebaseCharacterID', 'FreebaseActorID']

# Scraping

## Scraping Imbd

In [8]:
import imdb_scraper as imdb

### Example on how to use the scraper

<b><span style="color:red">Don't close the browser when it is in use </span></b>

In [9]:
# initialize the scraper object
myscraper = imdb.ImdbScraper()

# select a movie wikipedia id
movie_id = movies.iloc[74623]["WikiID"]

# scrape the movie infos
scraped_data = myscraper.get_imdb_infos(movie_id)

# close the browser
myscraper.close()

scraped_data

{'global_revenue': None,
 'budget': 2846000.0,
 'gross_domestic': None,
 'opening_weekend': None,
 'rating_score': 6.7,
 'number_of_ratings': 1700.0,
 'watched_rank': None,
 'producer': 'Robert Wise',
 'release_year': None}

### Let the scraping begin!

Computing `n_computer` partitions to parallelize ImDB's scraping on multiple computers

In [10]:
n           = movies.shape[0]
n_computer  = 6
size        = n//n_computer

# create a uniform partition of indices from 0 to n-1 for n_computer computers
indices         = [i for i in range(n)]
partitions      = [indices[i*size:(i+1)*size] for i in range(n_computer)]
partitions[-1]  = partitions[-1] + indices[n_computer*size:] 

In [11]:
# show first and last element of each partition
partition_intervals = [(partitions[i][0], partitions[i][-1]) for i in range(n_computer)]
print(f"number of elements: {n}")
print(f"partitions' size: {size}\n")
for i in range(len(partition_intervals)):
    print(f"partition {i}: {partition_intervals[i]}")

number of elements: 81741
partitions' size: 13623

partition 0: (0, 13622)
partition 1: (13623, 27245)
partition 2: (27246, 40868)
partition 3: (40869, 54491)
partition 4: (54492, 68114)
partition 5: (68115, 81740)


Each user should use its index to compute his attributed dataset

In [12]:
"""
Select your index here:
    - Anthony   0 & 1
    - Anton     2
    - Aymeric   3
    - Eric      4
    - Yara      5
"""
index = 2

Since ImDB has a protection against bots, using `requests` yields a `forbidden` error. The use of `selenium` to simmulate a real human operator avoids this problem and allows to scrape, though more slowly.

In [10]:
def scrape_partition(index):
    # create a header dataframe
    header = pd.DataFrame(columns=["WikiID", "Name", "global_revenue", "budget", "gross_domestic", "opening_weekend", 
                               "rating_score", "number_of_ratings", "watched_rank", "producer", "release_year"])  
    
    # csv file name
    csv_file = 'MovieSummaries/imdb_scraped_data_' + str(index) + '.csv'
    
    #initialize the scraper object
    myscraper = imdb.ImdbScraper()
    
    # if the csv_file doesn't exist, create it and write the header
    if not os.path.isfile(csv_file):
        header.to_csv(csv_file, index=False)
        starting_index = 0
    else:
        scraped = pd.read_csv(csv_file)
        starting_index = scraped.shape[0]
       
    index_range = partitions[index][starting_index:]
    
    for i in index_range:
        movie_id = movies.iloc[i]["WikiID"]
    
        movie_infos = myscraper.get_imdb_infos(movie_id)
        row = [movie_id, movies["Name"].values[i], *list(movie_infos.values())]
    
        # create a new DataFrame for the row
        row_df = pd.DataFrame([row], columns=header.columns)
    
        # replace None values with the string 'None'
        row_df = row_df.fillna('None')
    
        # append the row DataFrame to the CSV file
        row_df.to_csv(csv_file, mode='a', header=False, index=False)
    
    myscraper.close()

In [14]:
scrape_partition(index)

## Parallelized version

In [11]:
n           = movies.shape[0]
n_threads  = 8
size        = n//n_threads

# create a uniform partition of indices from 0 to n-1 for n_threads computers
indices         = [i for i in range(n)]
partitions      = [indices[i*size:(i+1)*size] for i in range(n_threads)]
partitions[-1]  = partitions[-1] + indices[n_threads*size:] 

In [12]:
# show first and last element of each partition
partition_intervals = [(partitions[i][0], partitions[i][-1]) for i in range(n_threads)]
print(f"number of elements: {n}")
print(f"partitions' size: {size}\n")
for i in range(len(partition_intervals)):
    print(f"partition {i}: {partition_intervals[i]}")

number of elements: 81741
partitions' size: 4087

partition 0: (0, 4086)
partition 1: (4087, 8173)
partition 2: (8174, 12260)
partition 3: (12261, 16347)
partition 4: (16348, 20434)
partition 5: (20435, 24521)
partition 6: (24522, 28608)
partition 7: (28609, 32695)
partition 8: (32696, 36782)
partition 9: (36783, 40869)
partition 10: (40870, 44956)
partition 11: (44957, 49043)
partition 12: (49044, 53130)
partition 13: (53131, 57217)
partition 14: (57218, 61304)
partition 15: (61305, 65391)
partition 16: (65392, 69478)
partition 17: (69479, 73565)
partition 18: (73566, 77652)
partition 19: (77653, 81740)


In [ ]:
import threading

threads = []

for i in range(n_threads):
    threads.append(threading.Thread(target=scrape_partition, args=(i,)))

for thread in threads:
    thread.start()

for thread in threads:
    thread.join()

## Merging the scraped partitions

In [36]:
csv_file = r"./MovieSummaries/imdb_scraped_dataset.csv"
try:
    partition_dataset = pd.read_csv(csv_file)
except:
    partition_file = r'partitions/imdb_scraped_data_'
    partition_dataset = pd.read_csv(partition_file + '0.csv')

    for i in range(1, n_computer):
        partition = partition_file + str(i) + '.csv'
        partition_dataset = pd.concat([partition_dataset, pd.read_csv(partition)], axis=0)
        partition_dataset.to_csv(csv_file, index=False)

"\nfor i in range(1, n_computer):\n    partition = partition_file + str(i) + '.csv'\n    part_df = pd.concat([part_df, pd.read_csv(partition)], axis=0)\n"

<br><br>

---

## TMBD API

In [12]:
import tmdb_scraper

In [13]:
# example
my_tmdb_scraper = tmdb_scraper.tmdb_scraper()
my_tmdb_scraper.get_tmdb_infos("Star Wars : Episode IV - A New Hope")

{'tmdb_id': 11,
 'movie_name': 'Star Wars : Episode IV - A New Hope',
 'release_date': '1977-05-25',
 'revenue': 775398007,
 'budget': 11000000,
 'rating': 8.204,
 'vote_count': 19278,
 'popularity': 86.431,
 'runtime': 121,
 'production': None}

In [12]:
dataset_names   = movies["Name"].values

# create an empty DataFrame for the first row
header = pd.DataFrame(columns=["tmdb_id", "movie_name", "release_date", "revenue", "budget", "rating", "vote_count", "popularity", "runtime", "production"])

csv_file = 'MovieSummaries/tmbd_scraped_data.csv'

# if the csv_file doesn't exist yet, make a header
if not os.path.isfile(csv_file):
    # write the DataFrame to a CSV file
    header.to_csv(csv_file, index=False)

index_range = np.arange(0, 10)

for i in index_range:
    name = dataset_names[i]

    movie_infos = my_tmdb_scraper.get_tmdb_infos(name)
    row = [*list(movie_infos.values())]

    # create a new DataFrame for the row
    row_df = pd.DataFrame([row], columns=header.columns)

    # replace None values with the string 'None'
    row_df = row_df.fillna('None')

    # append the row DataFrame to the CSV file
    row_df.to_csv(csv_file, mode='a', header=False, index=False)

    #print(f"{name}  : {movie_infos}")

# Testing zone

## Wikipedia API

In [243]:
import wikipediaapi
import requests

In [250]:
resp = requests.get("https://en.wikipedia.org/w/api.php?action=query&pageids=975900&format=json")

In [251]:
resp.json()

{'batchcomplete': '',
 'query': {'pages': {'975900': {'pageid': 975900,
    'ns': 0,
    'title': 'Ghosts of Mars'}}}}

## TMDB

In [10]:
import requests
import tmdbsimple as tmdb

In [11]:
movie_name = "My Friend Ganesha"

In [12]:
tmdb.REQUESTS_SESSION = requests.Session()
search      = tmdb.Search()

search.movie(query=movie_name)
tmdb_id     = search.results[0]['id']
result      = tmdb.Movies(tmdb_id).info()

revenue     = result['revenue']
budget      = result['budget']
rating      = result['vote_average']
vote_count  = result['vote_count']
popularity  = result['popularity']
runtime     = result['runtime']
production  = result['production_companies'][0]['name']
release_date = result['release_date']

IndexError: list index out of range

## IMDB

In [225]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re

In [226]:
movie_name = dataset_names[0]
movie_name

'Ghosts of Mars 2001-08-24'

In [227]:
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=chrome_options)

# access the webpage
driver.get("https://www.imdb.com/")

# find the search bar & search button
search_bar = driver.find_element("xpath", '//*[@id="suggestion-search"]')
search_button = driver.find_element("xpath", '//*[@id="suggestion-search-button"]')

# search for the movie
search_bar.send_keys(movie_name)
search_button.click()


In [228]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [230]:
dates = soup.find_all("span", {"class": "ipc-metadata-list-summary-item__li"})

In [235]:
dates

[<span aria-disabled="false" class="ipc-metadata-list-summary-item__li">2001</span>,
 <span aria-disabled="false" class="ipc-metadata-list-summary-item__li">Natasha Henstridge, Ice Cube</span>,
 <span aria-disabled="false" class="ipc-metadata-list-summary-item__li">2001</span>,
 <span aria-disabled="false" class="ipc-metadata-list-summary-item__li">Vidéo</span>,
 <span aria-disabled="false" class="ipc-metadata-list-summary-item__li">John Carpenter</span>,
 <span aria-disabled="false" class="ipc-metadata-list-summary-item__li">2001</span>,
 <span aria-disabled="false" class="ipc-metadata-list-summary-item__li">Vidéo</span>,
 <span aria-disabled="false" class="ipc-metadata-list-summary-item__li">Anthrax, Tiago Becker</span>,
 <span aria-disabled="false" class="ipc-metadata-list-summary-item__li">2001</span>,
 <span aria-disabled="false" class="ipc-metadata-list-summary-item__li">Vidéo</span>,
 <span aria-disabled="false" class="ipc-metadata-list-summary-item__li">Danielle Burgio, John Ca

In [238]:
movies.iloc[10]

WikiID                                                    175026
FreebaseID                                             /m/017n1p
Name                                               Sarah and Son
ReleaseDate                                                 1930
Revenue                                                      NaN
Runtime                                                     86.0
Languages                     {"/m/02h40lc": "English Language"}
Countries              {"/m/09c7w0": "United States of America"}
Genres         {"/m/07s9rl0": "Drama", "/m/01g6gs": "Black-an...
Name: 10, dtype: object

In [ ]:

# find the first result
first_result = driver.find_element("xpath", '//*[@id="__next"]/main/div[2]/div[3]/section/div/div[1]/section[2]/div[2]/ul/li[1]')
first_result.click()

#parse the resulting webpage
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
box_office = soup.find('div', {'data-testid': "title-boxoffice-section"})
try:
    brut_li                 = box_office.find('li', {'data-testid' : "title-boxoffice-cumulativeworldwidegross"})
    global_revenue_raw      = brut_li.find('span', {'class' : "ipc-metadata-list-item__list-content-item"}).text
    global_revenue          = int(re.findall(r'\d+', global_revenue_raw.replace('\u202f', ''))[0])
except:
    global_revenue          = None

try:
    budget_li               = box_office.find('li', {'data-testid' : "title-boxoffice-budget"})
    budget_raw              = budget_li.find('span', {'class' : "ipc-metadata-list-item__list-content-item"}).text
    budget                  = int(re.findall(r'\d+', budget_raw.replace('\u202f', ''))[0])
except:
    budget = None

try:
    gross_domestic_li       = box_office.find('li', {'data-testid' : "title-boxoffice-grossdomestic"})
    gross_domestic_raw      = gross_domestic_li.find('span', {'class' : "ipc-metadata-list-item__list-content-item"}).text
    gross_domestic          = int(re.findall(r'\d+', gross_domestic_raw.replace('\u202f', ''))[0])
except:
    gross_domestic          = None

try:
    opening_weekend_li      = box_office.find('li', {'data-testid' : "title-boxoffice-openingweekenddomestic"})
    opening_weekend_raw     = opening_weekend_li.find('span', {'class' : "ipc-metadata-list-item__list-content-item"}).text
    opening_weekend         = int(re.findall(r'\d+', opening_weekend_raw.replace('\u202f', ''))[0])
except:
    opening_weekend         = None

try:
    rating_score_div        = soup.find('div', {'data-testid' : "hero-rating-bar__aggregate-rating__score"})
    rating_score_raw        = rating_score_div.text
    rating_score            = float(re.findall(r'\d{1}.\d{1}', rating_score_raw.replace(',', '.').replace('\u202f', ''))[0])
except:
    rating_score            = None

try:
    number_of_ratings_div   = soup.find('div', {'data-testid' : "hero-rating-bar__aggregate-rating"})
    number_of_ratings_raw   = number_of_ratings_div.find('div', {'class' : "sc-bde20123-3 bjjENQ"}).text
    number_of_ratings       = float(re.findall(r'\d+.?\d?', number_of_ratings_raw.replace(',', '.').replace('\u202f', ''))[0])

    rating_unit             = number_of_ratings_raw[-1]
    if (rating_unit=='M'):
        number_of_ratings   = 1000000*number_of_ratings
    elif (rating_unit=='k'):
        number_of_ratings   = 1000*number_of_ratings
except:
    number_of_ratings       = None

try:
    watched_rank_div        = soup.find('div', {'data-testid' : "hero-rating-bar__popularity"})
    watched_rank_raw        = watched_rank_div.find('div', {'data-testid' : "hero-rating-bar__popularity__score"}).text
    watched_rank            = int(re.findall(r'\d+', watched_rank_raw.replace('\u202f', ''))[0])
except:
    watched_rank            = None

try:
    cast_section            = soup.find('section', {"data-testid" : "title-cast"})
    cast_ul                 = cast_section.find('ul', {"class" : "ipc-metadata-list ipc-metadata-list--dividers-all sc-bfec09a1-8 iiDmgX ipc-metadata-list--base"})
    cast_raw                = cast_ul.find('a', {'class' : 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'}).text
    producer               = cast_raw
except:
    producer                = None

In [ ]:
producer

## Wikipedia stuff

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
movie_id = movies.iloc[5]["WikiID"]
resp = requests.get(f"http://en.wikipedia.org/w/api.php?action=query&prop=info&pageids={movie_id}&inprop=url&format=json")
wikipedia_movie_link = resp.json()['query']['pages'][str(movie_id)]['fullurl']

In [ ]:
resp.json()

{'batchcomplete': '',
 'query': {'pages': {'13696889': {'pageid': 13696889,
    'ns': 0,
    'title': 'The Gangsters',
    'contentmodel': 'wikitext',
    'pagelanguage': 'en',
    'pagelanguagehtmlcode': 'en',
    'pagelanguagedir': 'ltr',
    'touched': '2023-11-07T16:37:20Z',
    'lastrevid': 1112156191,
    'length': 1639,
    'fullurl': 'https://en.wikipedia.org/wiki/The_Gangsters',
    'editurl': 'https://en.wikipedia.org/w/index.php?title=The_Gangsters&action=edit',
    'canonicalurl': 'https://en.wikipedia.org/wiki/The_Gangsters'}}}}

In [ ]:
wikipedia_movie_link

'https://en.wikipedia.org/wiki/The_Gangsters'

In [ ]:
html = requests.get(wikipedia_movie_link)

In [ ]:
# parse wikipedia_movie_link
soup = BeautifulSoup(html.text, 'html.parser')
tools = soup.find_all('div', {'id': 'vector-page-tools'})
wikidata_link = tools[0].find('li', {'id':'t-wikibase'}).find('a')['href']

wikidata_html = requests.get(wikidata_link)

In [ ]:
new_soup = BeautifulSoup(wikidata_html.text, 'html.parser')
wiki_imdb = new_soup.find('div', {'id': 'P345'})
imdb_id = wiki_imdb.find('div', {'class': 'wikibase-snakview-value wikibase-snakview-variation-valuesnak'}).text
imdb_link = "https://www.imdb.com/title/" + str(imdb_id) + "/"
imdb_link

'https://www.imdb.com/title/tt0002894/'

In [ ]:
imdb_link

'https://wikidata-externalid-url.toolforge.org/?p=345&url_prefix=https://www.imdb.com/&id=tt0002894'

---

In [ ]:
import imdb_scraper as imdb
from bs4 import BeautifulSoup
import requests

In [ ]:
# example of how to use the scraper
myscraper = imdb.ImdbScraper()

In [ ]:

movie_id = movies.iloc[24975]["WikiID"]
myscraper.get_imdb_infos(movie_id)

imdb_html = myscraper.driver.page_source
soup = BeautifulSoup(imdb_html, 'html.parser')

soup.find('div', {'class': 'sc-dffc6c81-0 grcyBP'}).find('a', {'class': 'ipc-link ipc-link--baseAlt ipc-link--inherit-color'}).text